# SageMaker Pipelines Customer Churn Prediction

Amazon SageMaker Model Building Pipelines offers machine learning (ML) application developers and operations engineers the ability to orchestrate SageMaker jobs and author reproducible ML pipelines. It also enables them to deploy custom-build models for inference in real-time with low latency, run offline inferences with Batch Transform, and track lineage of artifacts. They can institute sound operational practices in deploying and monitoring production workflows, deploying model artifacts, and tracking artifact lineage through a simple interface, adhering to safety and best practice paradigms for ML application development.

The SageMaker Pipelines service supports a SageMaker Pipeline domain specific language (DSL), which is a declarative JSON specification. This DSL defines a directed acyclic graph (DAG) of pipeline parameters and SageMaker job steps. The SageMaker Python Software Developer Kit (SDK) streamlines the generation of the pipeline DSL using constructs that engineers and scientists are already familiar with.

## SageMaker Pipelines

SageMaker Pipelines supports the following activities, which are demonstrated in this notebook:

* Pipelines - A DAG of steps and conditions to orchestrate SageMaker jobs and resource creation.
* Processing job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* Training job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* Conditional execution steps - A step that provides conditional execution of branches in a pipeline.
* Register model steps - A step that creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* Create model steps - A step that creates a model for use in transform steps or later publication as an endpoint.
* Clarify steps - A ClarifyCheck step that conduct model explainability check which launches a processing job that runs the SageMaker Clarify prebuilt container.
* Parametrized Pipeline executions - Enables variation in pipeline executions according to specified parameters.

## Notebook Overview

This notebook shows how to:

* Define a set of Pipeline parameters that can be used to parametrize a SageMaker Pipeline.
* Define a Processing step that extracts data from feature store to create the train, validation and test data sets.
* Define a Training step that trains a model on the preprocessed train data set.
* Define a Processing step that evaluates the trained model's performance on the test dataset.
* Define a Create Model step that creates a model from the model artifacts used in training.
* Define a Clarify check step that performs model explainability check.
* Define a Conditional step that measures a condition based on output from prior steps and conditionally executes other steps.
* Define a Register Model step that creates a model package from the estimator and model artifacts used to train the model.
* Define and create a Pipeline definition in a DAG, with the defined parameters and steps.
* Start a Pipeline execution and wait for execution to complete.
* Download the model evaluation report from the S3 bucket for examination.


## A SageMaker Pipeline

The pipeline that you create follows a typical machine learning (ML) application pattern of preprocessing, training, evaluation, model creation, batch transformation, and model registration:


## 1. Preparation

**Required IAM Role**
- IAM
- Lambda

**Let's start by specifying:**

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [ ]:
import json
import os

import boto3
import sagemaker
import sagemaker.session
from sagemaker import get_execution_role

from typing import Dict

import sagemaker
from sagemaker.dataset_definition.inputs import (
    AthenaDatasetDefinition,
    DatasetDefinition,
)
from sagemaker.workflow.clarify_check_step import (
    DataBiasCheckConfig,
    ClarifyCheckStep,
    ModelBiasCheckConfig,
    ModelExplainabilityCheckConfig,
    ClarifyCheckConfig,
)
from sagemaker.clarify import (
    DataConfig,
    BiasConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)
from sagemaker.drift_check_baselines import DriftCheckBaselines
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.callback_step import (
    CallbackOutput,
    CallbackOutputTypeEnum,
    CallbackStep,
)
from sagemaker.workflow.lambda_step import (
    LambdaOutput,
    LambdaOutputTypeEnum,
    LambdaStep,
)
from sagemaker.inputs import TrainingInput
from sagemaker.lambda_helper import Lambda
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    Processor,
    ScriptProcessor,
)
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.clarify_check_step import ClarifyCheckStep, DataBiasCheckConfig
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.functions import Join
from sagemaker.workflow.parameters import (
    ParameterFloat,
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.pipeline import Pipeline

from sagemaker.workflow.step_collections import EstimatorTransformer, RegisterModel
from sagemaker.workflow.steps import CacheConfig, ProcessingStep, Step, TrainingStep
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.functions import JsonGet
from sagemaker.utils import name_from_base

import uuid

In [ ]:
role = get_execution_role()

region = boto3.Session().region_name

sagemaker_session = sagemaker.Session()
account = sagemaker_session.boto_session.client("sts").get_caller_identity()["Account"]

bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-customer-churn-connect'
base_job_prefix = 'Demo-xgboost-churn-connect'

upload data to s3 input location

In [ ]:
s3_client = boto3.client("s3")
s3_client.upload_file('./data/churn_processed.csv', bucket, prefix+'/input/churn_processed.csv')
inputDir = f"s3://{bucket}/{prefix}/input/"

## 2. Define Parameters to Parametrize Pipeline Execution

Define Pipeline parameters that you can use to parametrize the pipeline. Parameters enable custom pipeline executions and schedules without having to modify the Pipeline definition.

The supported parameter types include:

* `ParameterString` - represents a `str` Python type
* `ParameterInteger` - represents an `int` Python type
* `ParameterFloat` - represents a `float` Python type

These parameters support providing a default value, which can be overridden on pipeline execution. The default value specified should be an instance of the type of the parameter.

The parameters defined in this workflow include:

* `processing_instance_type` - The `ml.*` instance type of the processing job.
* `processing_instance_count` - The instance count of the processing job.
* `training_instance_type` - The `ml.*` instance type of the training job.
* `train_instance_count` - The instance count of the training job.
* `model_approval_status` - What approval status to register the trained model with for CI/CD purposes ( "PendingManualApproval" is the default).
* `model_output` - The S3 bucket URI location of the model output path

**You must have run the previous sequential notebooks to retrieve variables using the StoreMagic command.**

In [ ]:
# parameters for pipeline execution

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount", default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.m5.xlarge"
)

train_instance_count = ParameterInteger(
    name="TrainingInstanceCount",
    default_value=1,
)
train_instance_type = ParameterString(
    name="TrainingInstance",
    default_value="ml.m5.xlarge",
)

model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval",
    enum_values=[
        "PendingManualApproval",
        "Approved",
    ],
)
model_output = ParameterString(
    name="ModelOutputUrl",
    default_value=f"s3://{bucket}/{prefix}/model",
)


Workflow for using glue with callback
```
Pipeline->Callback->SQS->Lambda->StepFunction->Job execution->Glue
                                                   |
                                                   StatusCheck->Pipeline
```

## 3. SageMaker Pipelines preparation

### 3.1 Create lambda functions
Prepare role and function names

In [ ]:
from setup_iam_roles import create_lambda_sm_pipeline_role

lambda_role_name = 'lambda_sm_pipeline_role'
lambda_role_arn = create_lambda_sm_pipeline_role(
    role_name=lambda_role_name
)

with open('lambdas/execute_sfn_function.zip', 'rb') as f:
    zipped_code = f.read()

exec_sfn_func_name = "execute_sfn_function"
exec_glue_func_name = "execute_glue_job_function"
check_glue_func_name = "check_glue_job_function"

lambda_client = boto3.client('lambda')

Prepare zip files

In [ ]:
import zipfile

# Function launching stepfunction
with zipfile.ZipFile('lambdas/execute_sfn_function.zip', 'w') as archive:
    archive.write('lambdas/execute_function.py', 'execute_function.py')

# Function starting glue job
with zipfile.ZipFile('lambdas/execute_glue_job_function.zip', 'w') as archive:
    archive.write('lambdas/execute_glue_job.py', 'execute_glue_job.py')

# Function checking the status of glue job
with zipfile.ZipFile('lambdas/check_glue_job_function.zip', 'w') as archive:
    archive.write('lambdas/check_glue_job.py', 'check_glue_job.py')

Create lambda for executing glue job

In [ ]:
response = lambda_client.create_function(
    FunctionName=exec_sfn_func_name,
    Runtime='python3.8',
    Role=lambda_role_arn,
    Handler='execute_function.lambda_handler',
    Code=dict(ZipFile=zipped_code),
    Timeout=300, # Maximum allowable timeout
)

exec_sfn_arn = response["FunctionArn"]

Create function for executing glue job

In [ ]:
with open('lambdas/execute_glue_job_function.zip', 'rb') as f:
    zipped_code = f.read()

response = lambda_client.create_function(
    FunctionName=exec_glue_func_name,
    Runtime='python3.8',
    Role=lambda_role_arn,
    Handler='execute_glue_job.lambda_handler',
    Code=dict(ZipFile=zipped_code),
    Timeout=300, # Maximum allowable timeout
)

exec_glue_arn = response["FunctionArn"]

Create function for executing glue job check

In [ ]:
with open('lambdas/check_glue_job_function.zip', 'rb') as f:
    zipped_code = f.read()

response = lambda_client.create_function(
    FunctionName=check_glue_func_name,
    Runtime='python3.8',
    Role=lambda_role_arn,
    Handler='check_glue_job.lambda_handler',
    Code=dict(ZipFile=zipped_code),
    Timeout=300, # Maximum allowable timeout
)

check_func_arn = response["FunctionArn"]

### 3.2 Create StepFunctions

In [ ]:
from setup_iam_roles import create_sfn_role

definition = {
  "Comment": "An example of the Amazon States Language using a choice state.",
  "StartAt": "execute_glue",
  "States": {
    "execute_glue": {
      "Type" : "Task",
      "Resource": exec_glue_arn,
      "Next": "wait_state"
    },
    "wait_state": {
      "Type": "Wait",
      "Seconds" : 15,
      "Next": "get_status_state"
    },
    "get_status_state": {
      "Type" : "Task",
      "Resource": check_func_arn,
      "Next": "check_job_state"
    },
    "fail_job": {
      "Type": "Fail",
      "Error": "DescribeJob returned FAILED",
      "Cause": "AWS Job Failed"
    },
    "succeed_state": {
      "Type": "Succeed"
    },
    "check_job_state": {
      "Type" : "Choice",
      "Choices": [
        {
          "Variable": "$.jobDetails.jobStatus",
          "StringEquals": "FAILED",
          "Next": "fail_job"
        },
        {
          "Variable": "$.jobDetails.jobStatus",
          "StringEquals": "SUCCEEDED",
          "Next": "succeed_state"
        }
      ],
      "Default": "wait_state"
    }
  }
}

sfn_role_name = "sfn_callback_role"
sfn_role_arn = create_sfn_role(
    role_name=sfn_role_name
)

sfn_client = boto3.client('stepfunctions')
response = sfn_client.create_state_machine(
    name='preprocessing',
    definition=json.dumps(definition),
    roleArn=sfn_role_arn,
    type='STANDARD',
    tracingConfiguration={
        'enabled': True
    }
)

In [ ]:
state_machine_arn = response["stateMachineArn"]

### 3.3 Create SQS queue for pipeline

In this step, we'll create the SQS queue that will be used by the CallbackStep inside SageMaker Pipeline steps. SageMaker Pipelines will put a token to this queue that will serve as a trigger for your Lambda function which will initiate the task to process your data.

In [ ]:
sqs_client = boto3.client('sqs')

queue_name = 'pipeline_callbacks_glue_prep'
try:
    response = sqs_client.create_queue(
        QueueName=queue_name,
            Attributes={
                'VisibilityTimeout': "600"
            })
except:
    print(f"Failed to create queue")

Format the queue URL to the same format we will need later on.

In [ ]:
queue_url = f"https://sqs.{region}.amazonaws.com/{account}/{queue_name}"

In [ ]:
queue_url

### 3.4 Create ETL job

In [ ]:
from setup_iam_roles import create_glue_pipeline_role

glue_client = boto3.client('glue')

glue_role_name = "glue_pipeline_role"
glue_job_name = "preprocessing_customer_churn_glue"

glue_role_arn = create_glue_pipeline_role(role_name=glue_role_name, bucket=bucket)

In [ ]:
s3_client.upload_file('glue_preprossing.py', bucket, prefix+'/pipeline/glue_preprossing.py')
glue_script_location = f"s3://{bucket}/{prefix}/pipeline/glue_preprossing.py"
glue_script_location

In [ ]:
response = glue_client.create_job(
    Name=glue_job_name,
    Description='Prepare data for SageMaker training',
    Role=glue_role_arn,
    ExecutionProperty={
        'MaxConcurrentRuns': 2
    },
    Command={
        'Name': 'glueetl',
        'ScriptLocation': glue_script_location,
    },
    DefaultArguments={
        "--job-bookmark-option": "job-bookmark-enable",
        "--enable-metrics": "",
        "--additional-python-modules": "pyarrow==2,awswrangler==2.9.0,fsspec==0.7.4"
    },
    MaxRetries=0,
    Timeout=60,
    MaxCapacity=10.0,
    GlueVersion='2.0'
)
glue_job_name = response['Name']

In [ ]:
glue_job_name

### 3.5 Set up Lambda as SQS target
Next, we'll attach the lambda function created above to the SQS queue we previously created. This ensures that your Lambda will be triggered when new messages are put to your SQS queue.

In [ ]:
evt_source_mapping_response = lambda_client.create_event_source_mapping(
    EventSourceArn=f'arn:aws:sqs:{region}:{account}:{queue_name}',
    FunctionName='execute_sfn_function',
    Enabled=True,
    BatchSize=10
)

## 4. Pipeline Steps
### 4.1 Callback Step
First, we'll configure the callback step.

The callback step will accept the following inputs:

- S3 location of our raw taxi data
- SQS queue

The callback step will return the following outputs:

- S3 location of processed data to be used for model training

In [ ]:
from sagemaker.workflow.callback_step import CallbackStep,CallbackOutput,CallbackOutputTypeEnum

cb_train_uri=CallbackOutput(output_name="trainUri", output_type=CallbackOutputTypeEnum.String)
cb_validation_uri=CallbackOutput(output_name="validationUri", output_type=CallbackOutputTypeEnum.String)
cb_test_uri=CallbackOutput(output_name="testUri", output_type=CallbackOutputTypeEnum.String)

step_callback_data = CallbackStep(
                    name="GluePrepCallbackStep",
                    sqs_queue_url=queue_url,
                    inputs={
                        "targetJob": glue_job_name,
                        "processedDir": f"s3://{bucket}/{prefix}/processed/",
                        "inputDir": inputDir,
                        "stateMachineArn": state_machine_arn
                    },
                    outputs=[
                        cb_train_uri,
                        cb_validation_uri,
                        cb_test_uri
                    ],
                )

### 4.2 Define a TrainStep

#### Define a Training Step to Train a Model

In this section, use Amazon SageMaker's [XGBoost Algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) to train on this dataset. Configure an Estimator for the XGBoost algorithm and the input dataset. A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_output` so that it can be hosted later. 

The model path where the models from training will be saved is also specified.

Note the `training_instance_type` parameter may be used in multiple places in the pipeline. In this case, the `training_instance_type` and `train_instance_count` is passed into the estimator.

In [ ]:
# training step for generating model artifacts
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
)
xgb_train = sagemaker.estimator.Estimator(
    image_uri=image_uri,
    instance_type=train_instance_type,
    instance_count=train_instance_count,
    output_path=model_output,
    base_job_name=f"{base_job_prefix}-train",
    sagemaker_session=sagemaker_session,
    role=role,
)

# Set some hyper parameters
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
xgb_train.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    silent=0,
    objective="binary:logistic",
    num_round=100,
    eval_metric='auc'
)


Finally, use the estimator instance to construct a `TrainingStep` as well as the `properties` of the prior `ProcessingStep` used as input in the `TrainingStep` inputs and the code that's executed when the pipeline invokes the pipeline execution. This is similar to an estimator's `fit` method in the Python SDK.

Pass in the `S3Uri` of the `"train"` output channel to the `TrainingStep`. Also, use the other `"validation"` output channel for model evaluation in the pipeline. The `properties` attribute of a Pipeline step matches the object model of the corresponding response of a describe call. These properties can be referenced as placeholder values and are resolved at runtime. For example, the `ProcessingStep` `properties` attribute matches the object model of the [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response object.

In [ ]:
step_train = TrainingStep(
    name="TrainModel",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data=cb_train_uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=cb_validation_uri,
            content_type="text/csv",
        ),
    },

)

### 4.3 Define a Model Evaluation Step to Evaluate the Trained Model

First, develop an evaluation script that is specified in a Processing step that performs the model evaluation.

After pipeline execution, you can examine the resulting `evaluation.json` for analysis.

The evaluation script uses `xgboost` to do the following:

* Load the model.
* Read the test data.
* Issue predictions against the test data.
* Build a classification report, including mae, mse, rmse and r2 metrics.
* Save the evaluation report to the evaluation directory.

Next, create an instance of a `ScriptProcessor` processor and use it in the `ProcessingStep`.

Note the `processing_instance_type` parameter passed into the processor.

In [ ]:
# processing step for evaluation
script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name=f"{base_job_prefix}/script-eval",
    sagemaker_session=sagemaker_session,
    role=role,
)


Use the processor instance to construct a `ProcessingStep`, along with the input and output channels and the code that will be executed when the pipeline invokes pipeline execution. This is similar to a processor instance's `run` method in the Python SDK.

Specifically, the `S3ModelArtifacts` from the `step_train` `properties` and the `S3Uri` of the `"test"` output channel of the `step_process` `properties` are passed into the inputs. The `TrainingStep` and `ProcessingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) and  [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response objects, respectively.

In [ ]:
evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json",
)
eval_output = f"s3://{bucket}/{prefix}/Evaluation/output/evaluation.json"
step_eval = ProcessingStep(
    name="EvaluateModel",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
#             source=cb_test_uri,
            source=f"s3://{bucket}/{prefix}/processed/test/test.csv",
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name="evaluation", 
            source="/opt/ml/processing/evaluation",
            destination=eval_output
        ),
    ],
    code="evaluation.py",
    property_files=[evaluation_report],

)

### 4.4 Define a Create Model Step to Create a Model

In order to perform batch transformation using the example model, create a SageMaker model. 

Specifically, pass in the `S3ModelArtifacts` from the `TrainingStep`, `step_train` properties. The `TrainingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) response object.

In [ ]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.model import Model

model = Model(
    image_uri=image_uri,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker_session,
    role=role,
)

inputs = CreateModelInput(
    instance_type="ml.m5.xlarge",
)
step_create_model = CreateModelStep(
    name="CustomerChurnCreateModel",
    model=model,
    inputs=inputs,
)

### 4.6 Define a Register Model Step to Create a Model Package

Use the estimator instance specified in the training step to construct an instance of `RegisterModel`. The result of executing `RegisterModel` in a pipeline is a model package. A model package is a reusable model artifacts abstraction that packages all ingredients required for inference. Primarily, it consists of an inference specification that defines the inference image to use along with an optional model weights location.

A model package group is a collection of model packages. A model package group can be created for a specific ML business problem, and new versions of the model packages can be added to it. Typically, customers are expected to create a ModelPackageGroup for a SageMaker pipeline so that model package versions can be added to the group for every SageMaker Pipeline run.

The construction of `RegisterModel` is similar to an estimator instance's `register` method in the Python SDK.

Specifically, pass in the `S3ModelArtifacts` from the `TrainingStep`, `step_train` properties. The `TrainingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) response object.

Note that the specific model package group name provided in this notebook can be used in the model registry and CI/CD work with SageMaker Projects.

In [ ]:
# register model step that will be conditionally executed
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=eval_output,
        content_type="application/json",
    )
)
model_package_group_name = "CustomerChurnModelPackage"
step_register = RegisterModel(
    name="RegisterModel",
    estimator=xgb_train,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.t2.large", "ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)

### 4.7 Define a Condition Step to Check Accuracy and Conditionally Create a Model and Run a Batch Transformation and Register a Model in the Model Registry

In this step, the model is registered only if the accuracy of the model, as determined by the evaluation step `step_eval`, exceeded a specified value. A `ConditionStep` enables pipelines to support conditional execution in the pipeline DAG based on the conditions of the step properties. 

In the following section, you:

* Define a `ConditionLessThanOrEqualTo` on the accuracy value found in the output of the evaluation step, `step_eval`.
* Use the condition in the list of conditions in a `ConditionStep`.
* Pass the `CreateModelStep` and `TransformStep` steps, and the `RegisterModel` step collection into the `if_steps` of the `ConditionStep`, which are only executed, if the condition evaluates to `True`.

In [ ]:
# condition step for evaluating model quality and branching execution
cond_lte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.roc_auc.value",
    ),
    right=0.95,
)
step_cond = ConditionStep(
    name="CheckEvaluation",
    conditions=[cond_lte],
    if_steps=[step_create_model, step_register],
    else_steps=[],
)

### 4.8 Define a Pipeline of Parameters, Steps, and Conditions

In this section, combine the steps into a Pipeline so it can be executed.

A pipeline requires a `name`, `parameters`, and `steps`. Names must be unique within an `(account, region)` pair.

Note:

* All of the parameters used in the definitions must be present.
* Steps passed into the pipeline do not have to be listed in the order of execution. The SageMaker Pipeline service resolves the _data dependency_ DAG as steps for the execution to complete.
* Steps must be unique to across the pipeline step list and all condition step if/else lists.

In [ ]:
# pipeline instance
pipeline = Pipeline(
    name="Demo-customer-churn-pipeline",
    parameters=[
        processing_instance_count,
        processing_instance_type,
        train_instance_count,
        train_instance_type,
        model_approval_status,
        model_output
    ],
    steps=[step_callback_data,
           step_train,
           step_eval,
           step_cond],
    sagemaker_session=sagemaker_session,
)

### 4.9 Submit the pipeline to SageMaker and start execution

Submit the pipeline definition to the Pipeline service. The role passed in will be used by the Pipeline service to create all the jobs defined in the steps.

In [ ]:
pipeline.upsert(role_arn=role)

#### Pipeline Operations: Examining and Waiting for Pipeline Execution

Describe the pipeline execution.

In [ ]:
execution = pipeline.start()

In [ ]:
execution.describe()

In [ ]:
# Wait for the execution to complete.
execution.wait()

### 4.10 Examining the Evalution

Examine the resulting model evaluation after the pipeline completes. Download the resulting `evaluation.json` file from S3 and print the report.

In [ ]:
from pprint import pprint


evaluation_json = sagemaker.s3.S3Downloader.read_file("{}/evaluation.json".format(
    step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
))
pprint(json.loads(evaluation_json))

#### Lineage

Review the lineage of the artifacts generated by the pipeline.

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer


viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    display(viz.show(pipeline_execution_step=execution_step))
    time.sleep(5)

## 5. Clean up

In [ ]:
# Delete lambda functions
for func_name in [exec_sfn_func_name, exec_glue_func_name, check_glue_func_name]:
    lambda_client.delete_function(FunctionName=func_name)


# Delete event source mapping with SQS arn
lambda_client.delete_event_source_mapping(UUID=evt_source_mapping_response["UUID"])

# Delete glue job
glue_client.delete_job(JobName=glue_job_name)

# Delete sqs
sqs_client.delete_queue(QueueUrl=queue_url)

# Delete stepfunction
sfn_client.delete_state_machine(stateMachineArn=state_machine_arn)

# Delete pipeline
pipeline.delete()

# Delete model
model.delete_model()

# Delete artefacts
!aws s3 delete s3://$bucket/$prefix

# GitHub Resource
This demo is available on GitHub: https://github.com/aws-samples/real-time-churn-prediction-with-amazon-connect-and-amazon-sagemaker